### 5. Datos de prueba

In [ ]:
# Leemos los datos de prueba que provienen de un archivo tipo json de la API de meta (Instagram)
file_path2 = "C:/Users/lriverosq/Downloads/instagram_comments.media.csv"
bd_prueba = pd.read_csv(file_path2, sep = ",")
bd_prueba.head()

In [ ]:
imagenes_sin_video_url = bd_prueba[~bd_prueba['media_url'].str.contains("video", case=False, na=False)]
imagenes_sin_video_url.head()

In [ ]:
import requests
from PIL import Image
from io import BytesIO

def descargar_imagen(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return Image.open(BytesIO(response.content))
        else:
            return None
    except Exception as e:
        print(f"Error al descargar la URL {url}: {e}")
        return None

# Crear una nueva columna con las imágenes descargadas
bd_prueba['imagen'] = bd_prueba['media_url'].apply(descargar_imagen)

In [ ]:
print("Tamaño del dataframe:",len(bd_prueba))
print("Faltantes de lectura de URL:",bd_prueba['media_url'].isnull().sum())

In [ ]:
import cv2

def cargar_y_preprocesar_imagen_ipl(imagen_ipl):
    try:
        if len(imagen_ipl.shape) == 3:
            imagen_ipl = cv2.cvtColor(imagen_ipl, cv2.COLOR_BGR2GRAY)
            imagen_ipl = cv2.resize(imagen_ipl, (100, 100))
            imagen_normalizada = imagen_ipl / 255.0
            imagen_array = np.array(imagen_normalizada, dtype=np.float32)

        return imagen_array
    except Exception as e:
        print(f"Error al procesar la imagen IPL: {e}")
        return None

bd_prueba['imagen_cod'] = bd_prueba['imagen'].apply(cargar_y_preprocesar_imagen_ipl)

In [ ]:
def preprocesar_imagen(imagen_pil):
    try:
        # Verificar si la imagen es None
        if imagen_pil is None:
            raise ValueError("La imagen es None.")

        imagen = tf.keras.preprocessing.image.img_to_array(imagen_pil)

        if imagen.shape[-1] > 1:
            imagen = tf.image.rgb_to_grayscale(imagen)
        imagen = tf.image.resize(imagen, [100, 100])
        imagen = tf.keras.utils.img_to_array(imagen) # Cargar array
        imagen = imagen / 255

        return imagen
    except Exception as e:
        print(f"Error al preprocesar la imagen: {e}")
        return None

# Aplicar la función de preprocesamiento a la columna de imágenes
bd_prueba['imagen_cod'] = bd_prueba['imagen'].apply(preprocesar_imagen)


In [ ]:
print("Faltantes de lectura de URL:",bd_prueba['imagen_cod'].isnull().sum())

In [ ]:
bd_prueba = bd_prueba[bd_prueba['imagen_cod'].notnull()]  # Eliminar filas con valores None en la columna
print("Faltantes de lectura de URL:",bd_prueba['imagen_cod'].isnull().sum())

In [ ]:
X_test_arrays = np.array(bd_prueba['imagen_cod'].to_list())

In [ ]:
X_test_arrays

In [ ]:
predicciones = modelo4.predict(X_test_arrays)
clases_predichas = np.argmax(predicciones, axis=1)

In [ ]:
# Imagenes en escala de grises con su clasificación
plt.figure(figsize=(10, 10))

for i, (idx, row) in enumerate(bd_prueba.head(25).iterrows()):  # Máximo 25 imágenes
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_test_arrays[i], cmap=plt.cm.binary)
    plt.xlabel(clases_predichas[i])

plt.show()

In [ ]:
bd_prueba['clasificacion'] = clases_predichas

In [ ]:
plt.figure(figsize=(10, 10))

for i, (idx, row) in enumerate(bd_prueba.head(25).iterrows()):  # Máximo 25 imágenes
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(row['imagen_cod'], cmap=plt.cm.binary)
    plt.xlabel(row['clasificacion'])

plt.show()